In [1]:
import keras

In [2]:
import numpy as np
import pandas as pd
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import mlflow
from mlflow.models import infer_signature

In [3]:
data=pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep=";",
)
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [4]:
### split data

train, test = train_test_split(data, test_size=0.25, random_state=42)

In [5]:
train_x = train.drop(["quality"], axis=1).values
train_y = train[['quality']].values.ravel()

In [6]:
test_x = test.drop(["quality"], axis=1).values
test_y = test[['quality']].values.ravel()

In [7]:
train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size=0.20, random_state=42)

In [8]:
signature=infer_signature(train_x,train_y)

In [9]:
### ANN Model

def train_model(params, epochs, train_x, train_y, valid_x, valid_y, test_x, test_y):
    
    ###define normalization
    mean = np.mean(train_x, axis=0)
    var = np.var(train_x, axis=0)

    ### define model architecture
    model = keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean, variance=var),
            keras.layers.Dense(64, activation='relu'),
            keras.layers.Dense(1)    
        ]
    )
    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate=params['lr'], 
        momentum=params['momentum'],
    ),
        loss= "mean_squared_error",
        metrics=[keras.metrics.RootMeanSquaredError()]
    )
    
    ### train model
    with mlflow.start_run(nested=True):
        model.fit(train_x, train_y, validation_data=(valid_x, valid_y),
                  epochs=epochs,
                  batch_size=32)
        
        ###evaluate model
        eval_result = model.evaluate(valid_x, valid_y, batch_size=32)
        eval_rmse= eval_result[1]

        ### log parameters
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse", eval_rmse)

        ### lof the model (TF)
        mlflow.tensorflow.log_model(model, "model", signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model":model}

In [10]:
def objective(params):
    # MLflow will track the parameters and results for each run
    result = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y,
    )
    return result

In [11]:
space = {
    "lr": hp.uniform("lr", 0.01, 0.1),
    "momentum": hp.uniform("momentum", 0.1, 0.9),
}

In [12]:
mlflow.set_experiment("/wine-quality")
with mlflow.start_run():
    ### Conduct hyperparameter search with hyperopt
    trials = Trials()
    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials,
    )

        # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")

2025/03/16 13:12:15 INFO mlflow.tracking.fluent: Experiment with name '/wine-quality' does not exist. Creating a new experiment.


Epoch 1/3                                            

 1/92 ━━━━━━━━━━━━━━━━━━━━ 16s 185ms/step - loss: 28.6751 - root_mean_squared_error: 5.3549
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 4.9657 - root_mean_squared_error: 2.0969 - val_loss: 0.5490 - val_root_mean_squared_error: 0.7410

Epoch 2/3                                            

 1/92 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.4892 - root_mean_squared_error: 0.6995
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 641us/step - loss: 0.5483 - root_mean_squared_error: 0.7401 - val_loss: 0.5646 - val_root_mean_squared_error: 0.7514

Epoch 3/3                                            

 1/92 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.6786 - root_mean_squared_error: 0.8238
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 640us/step - loss: 0.5424 - root_mean_squared_error: 0.7364 - val_loss: 0.5774 - val_root_mean_squared_error: 0.7599

 1/23 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.5110 - root_mean_squared_error: 0.7148
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 687us/s

### Inferencing


In [13]:
import mlflow

model_uri = 'runs:/890e12a6592e417cb55f26c021fad20c/model'

# Replace INPUT_EXAMPLE with your own input example to the model
# A valid input example is a data instance suitable for pyfunc prediction
input_data = test_x

# Verify the model with the provided input data using the logged dependencies.
# For more details, refer to:
# https://mlflow.org/docs/latest/models.html#validate-models-before-deployment
mlflow.models.predict(
    model_uri=model_uri,
    input_data=input_data,
    env_manager="uv",
)

/Users/juan.montenegro/Documents/mlflow/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025/03/16 13:20:51 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'
2025/03/16 13:20:51 INFO mlflow.utils.virtualenv: Creating a new environment in /var/folders/f1/812rtm0x4fd4p9bpmxb1wpx9r504b3/T/tmphuno29t0/envs/virtualenv_envs/mlflow-a75870b9310193ebfcb86a9fc66259f10c24cb11 with python version 3.10.0 using uv
Using CPython 3.10.0 interpreter at: /Users/juan.montenegro/Documents/mlflow/venv/bin/python3.10
Creating virtual environment at: /var/folders/f1/812rtm0x4fd4p9bpmxb1wpx9r504b3/T/tmphuno29t0/envs/virtualenv_envs/mlflow-a75870b9310193ebfcb86a9fc66259f10c24cb11
Activate with: source /var/folders/f1/812rtm0x4fd4p9bpmxb1wpx9r504b3/T/tmphuno29t0/envs/virtualenv_envs/m

ShellCommandException: Non-zero exit code: 1
Command: ['bash', '-c', 'source /var/folders/f1/812rtm0x4fd4p9bpmxb1wpx9r504b3/T/tmphuno29t0/envs/virtualenv_envs/mlflow-a75870b9310193ebfcb86a9fc66259f10c24cb11/bin/activate && uv pip install --prerelease=allow -r requirements.0505821202974212a1a7b7c6b8b92ec4.txt']

### Make predictions

In [ ]:
import mlflow
logged_model = 'runs:/890e12a6592e417cb55f26c021fad20c/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
import pandas as pd
loaded_model.predict(pd.DataFrame(data))